In [1]:
'''
图像增广处理汇总
'''

import cv2
import abc
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter, ImageOps, ImageDraw

def cv2pil(image):
    """
    将bgr格式的numpy的图像转换为pil
    :param image:   图像数组
    :return:    Image对象
    """
    assert isinstance(image, np.ndarray), 'input image type is not cv2'
    if len(image.shape) == 2:
        return Image.fromarray(image)
    elif len(image.shape) == 3:
        return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

def get_pil_image(image):
    """
    将图像统一转换为PIL格式
    :param image:   图像
    :return:    Image格式的图像
    """
    if isinstance(image, Image.Image):  # or isinstance(image, PIL.JpegImagePlugin.JpegImageFile):
        return image
    elif isinstance(image, np.ndarray):
        return cv2pil(image)

def get_cv_image(image):
    """
    将图像转换为numpy格式的数据
    :param image:   图像
    :return:    ndarray格式的图像数据
    """
    if isinstance(image, np.ndarray):
        return image
    elif isinstance(image, Image.Image):  # or isinstance(image, PIL.JpegImagePlugin.JpegImageFile):
        return pil2cv(image)

def pil2cv(image):
    """
    将Image对象转换为ndarray格式图像
    :param image:   图像对象
    :return:    ndarray图像数组
    """
    if len(image.split()) == 1:
        return np.asarray(image)
    elif len(image.split()) == 3:
        return cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
    elif len(image.split()) == 4:
        return cv2.cvtColor(np.asarray(image), cv2.COLOR_RGBA2BGR)

class TransBase(object):
    """
    数据增广的基类
    """

    def __init__(self, probability=1.):
        """
        初始化对象
        :param probability:     执行概率
        """
        super(TransBase, self).__init__()
        self.probability = probability

    @abc.abstractmethod
    def trans_function(self, _image):
        """
        初始化执行函数，需要进行重载
        :param _image:  待处理图像
        :return:    执行后的Image对象
        """
        pass

    # @utils.zlog
    def process(self, _image):
        """
        调用执行函数
        :param _image:  待处理图像
        :return:    执行后的Image对象
        """
        if np.random.random() < self.probability:
            return self.trans_function(_image)
        else:
            return _image

    def __call__(self, _image):
        """
        重载()，方便直接进行调用
        :param _image:  待处理图像
        :return:    执行后的Image
        """
        return self.process(_image)

class RandomContrast(TransBase):
    """
    随机对比度
    """

    def setparam(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, _image):
        _image = get_pil_image(_image)
        contrast_enhance = ImageEnhance.Contrast(_image)
        return contrast_enhance.enhance(random.uniform(self.lower, self.upper))

class RandomLine(TransBase):
    """
    在图像增加一条简单的随机线
    """

    def trans_function(self, image):
        image = get_pil_image(image)
        draw = ImageDraw.Draw(image)
        h = image.height
        w = image.width
        y0 = random.randint(h // 4, h * 3 // 4)
        y1 = np.clip(random.randint(-3, 3) + y0, 0, h - 1)
        color = random.randint(0, 30)
        draw.line(((0, y0), (w - 1, y1)), fill=(color, color, color), width=2)
        return image

class RandomBrightness(TransBase):
    """
    随机对比度
    """

    def setparam(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        image = get_pil_image(image)
        bri = ImageEnhance.Brightness(image)
        return bri.enhance(random.uniform(self.lower, self.upper))

class RandomColor(TransBase):
    """
    随机色彩平衡
    """

    def setparam(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        image = get_pil_image(image)
        col = ImageEnhance.Color(image)
        return col.enhance(random.uniform(self.lower, self.upper))

class RandomSharpness(TransBase):
    """
    随机锐度
    """

    def setparam(self, lower=0.5, upper=1.5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        image = get_pil_image(image)
        sha = ImageEnhance.Sharpness(image)
        return sha.enhance(random.uniform(self.lower, self.upper))

class Compress(TransBase):
    """
    随机压缩率，利用jpeg的有损压缩来增广
    """

    def setparam(self, lower=5, upper=85):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        img = get_cv_image(image)
        param = [int(cv2.IMWRITE_JPEG_QUALITY), random.randint(self.lower, self.upper)]
        img_encode = cv2.imencode('.jpeg', img, param)
        img_decode = cv2.imdecode(img_encode[1], cv2.IMREAD_COLOR)
        pil_img = cv2pil(img_decode)
        if len(image.split()) == 1:
            pil_img = pil_img.convert('L')
        return pil_img

class Exposure(TransBase):
    """
    随机区域曝光
    """

    def setparam(self, lower=5, upper=10):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        image = get_cv_image(image)
        h, w = image.shape[:2]
        x0 = random.randint(0, w)
        y0 = random.randint(0, h)
        x1 = random.randint(x0, w)
        y1 = random.randint(y0, h)
        transparent_area = (x0, y0, x1, y1)
        mask = Image.new('L', (w, h), color=255)
        draw = ImageDraw.Draw(mask)
        mask = np.array(mask)
        if len(image.shape) == 3:
            mask = mask[:, :, np.newaxis]
            mask = np.concatenate([mask, mask, mask], axis=2)
        draw.rectangle(transparent_area, fill=random.randint(150, 255))
        reflection_result = image + (255 - mask)
        reflection_result = np.clip(reflection_result, 0, 255)
        return cv2pil(reflection_result)

class Rotate(TransBase):
    """
    随机旋转
    """

    def setparam(self, lower=-5, upper=5):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."

    def trans_function(self, image):
        image = get_pil_image(image)
        rot = random.uniform(self.lower, self.upper)
        trans_img = image.rotate(rot, expand=True)
        return trans_img

class Blur(TransBase):
    """
    随机高斯模糊
    """

    def setparam(self, lower=0, upper=1):
        self.lower = lower
        self.upper = upper
        assert self.upper >= self.lower, "upper must be >= lower."
        assert self.lower >= 0, "lower must be non-negative."

    def trans_function(self, image):
        image = get_pil_image(image)
        image = image.filter(ImageFilter.GaussianBlur(radius=1.5))
        return image

class MotionBlur(TransBase):
    """
    随机运动模糊
    """

    def setparam(self, degree=5, angle=180):
        self.degree = degree
        self.angle = angle

    def trans_function(self, image):
        image = get_pil_image(image)
        angle = random.randint(0, self.angle)
        M = cv2.getRotationMatrix2D((self.degree / 2, self.degree / 2), angle, 1)
        motion_blur_kernel = np.diag(np.ones(self.degree))
        motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (self.degree, self.degree))
        motion_blur_kernel = motion_blur_kernel / self.degree
        image = image.filter(ImageFilter.Kernel(size=(self.degree, self.degree), kernel=motion_blur_kernel.reshape(-1)))
        return image

class Salt(TransBase):
    """
    随机椒盐噪音
    """

    def setparam(self, rate=0.02):
        self.rate = rate

    def trans_function(self, image):
        image = get_pil_image(image)
        num_noise = int(image.size[1] * image.size[0] * self.rate)
        # assert len(image.split()) == 1
        for k in range(num_noise):
            i = int(np.random.random() * image.size[1])
            j = int(np.random.random() * image.size[0])
            image.putpixel((j, i), int(np.random.random() * 255))
        return image

class AdjustResolution(TransBase):
    """
    随机分辨率
    """

    def setparam(self, max_rate=0.95, min_rate=0.5):
        self.max_rate = max_rate
        self.min_rate = min_rate

    def trans_function(self, image):
        image = get_pil_image(image)
        w, h = image.size
        rate = np.random.random() * (self.max_rate - self.min_rate) + self.min_rate
        w2 = int(w * rate)
        h2 = int(h * rate)
        image = image.resize((w2, h2))
        image = image.resize((w, h))
        return image

class Crop(TransBase):
    """
    随机抠图，并且抠图区域透视变换为原图大小
    """

    def setparam(self, maxv=2):
        self.maxv = maxv

    def trans_function(self, image):
        img = get_cv_image(image)
        h, w = img.shape[:2]
        org = np.array([[0, np.random.randint(0, self.maxv)],
                        [w, np.random.randint(0, self.maxv)],
                        [0, h - np.random.randint(0, self.maxv)],
                        [w, h - np.random.randint(0, self.maxv)]], np.float32)
        dst = np.array([[0, 0], [w, 0], [0, h], [w, h]], np.float32)
        M = cv2.getPerspectiveTransform(org, dst)
        res = cv2.warpPerspective(img, M, (w, h))
        return get_pil_image(res)

class Crop2(TransBase):
    """
    随机抠图，并且抠图区域透视变换为原图大小
    """

    def setparam(self, maxv_h=4, maxv_w=4):
        self.maxv_h = maxv_h
        self.maxv_w = maxv_w

    def trans_function(self, image_and_loc):
        image, left, top, right, bottom = image_and_loc
        w, h = image.size
        left = np.clip(left, 0, w - 1)
        right = np.clip(right, 0, w - 1)
        top = np.clip(top, 0, h - 1)
        bottom = np.clip(bottom, 0, h - 1)
        img = get_cv_image(image)
        try:
            res = get_pil_image(img[top:bottom, left:right])
            return res
        except AttributeError as e:
            print('error')
            image.save('test_imgs/t.png')
            print(left, top, right, bottom)

        h = bottom - top
        w = right - left
        org = np.array(
            [[left - np.random.randint(0, self.maxv_w), top + np.random.randint(-self.maxv_h, self.maxv_h // 2)],
             [right + np.random.randint(0, self.maxv_w), top + np.random.randint(-self.maxv_h, self.maxv_h // 2)],
             [left - np.random.randint(0, self.maxv_w), bottom - np.random.randint(-self.maxv_h, self.maxv_h // 2)],
             [right + np.random.randint(0, self.maxv_w), bottom - np.random.randint(-self.maxv_h, self.maxv_h // 2)]],
            np.float32)
        dst = np.array([[0, 0], [w, 0], [0, h], [w, h]], np.float32)
        M = cv2.getPerspectiveTransform(org, dst)
        res = cv2.warpPerspective(img, M, (w, h))
        return get_pil_image(res)

class Stretch(TransBase):
    """
    随机图像横向拉伸
    """

    def setparam(self, max_rate=1.2, min_rate=0.8):
        self.max_rate = max_rate
        self.min_rate = min_rate

    def trans_function(self, image):
        image = get_pil_image(image)
        w, h = image.size
        rate = np.random.random() * (self.max_rate - self.min_rate) + self.min_rate
        w2 = int(w * rate)
        image = image.resize((w2, h))
        return image

class DataAug:
    def __init__(self):
        self.crop = Crop(probability=0.1)
        self.crop2 = Crop2(probability=1.1)
        self.random_contrast = RandomContrast(probability=0.1)
        self.random_brightness = RandomBrightness(probability=0.1)
        self.random_color = RandomColor(probability=0.1)
        self.random_sharpness = RandomSharpness(probability=0.1)
        self.compress = Compress(probability=0.3)
        self.exposure = Exposure(probability=0.1)
        self.rotate = Rotate(probability=0.1)
        self.blur = Blur(probability=0.3)
        self.motion_blur = MotionBlur(probability=0.3)
        self.salt = Salt(probability=0.1)
        self.adjust_resolution = AdjustResolution(probability=0.1)
        self.stretch = Stretch(probability=0.1)
        self.random_line = RandomLine(probability=0.3)
        self.crop.setparam()
        self.crop2.setparam()
        self.random_contrast.setparam()
        self.random_brightness.setparam()
        self.random_color.setparam()
        self.random_sharpness.setparam()
        self.compress.setparam()
        self.exposure.setparam()
        self.rotate.setparam()
        self.blur.setparam()
        self.motion_blur.setparam()
        self.salt.setparam()
        self.adjust_resolution.setparam()
        self.stretch.setparam()

    def aug_img(self, img):
        img = self.crop.process(img)
        img = self.random_contrast.process(img)
        img = self.random_brightness.process(img)
        img = self.random_color.process(img)
        img = self.random_sharpness.process(img)
        img = self.random_line.process(img)

        if img.size[1] >= 32:
            img = self.compress.process(img)
            img = self.adjust_resolution.process(img)
            img = self.motion_blur.process(img)
            img = self.blur.process(img)
        img = self.exposure.process(img)
        img = self.rotate.process(img)
        img = self.salt.process(img)
        img = self.inverse_color(img)
        img = self.stretch.process(img)
        return img

    def inverse_color(self, image):
        if np.random.random() < 0.4:
            image = ImageOps.invert(image)
        return image

In [136]:
'''
Dataset 和 DataLoader
'''
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class RecTextLineDataset(Dataset):
    def __init__(self, config):
        """
        文本行 DataSet, 用于处理标注格式为 `img_path\tlabel` 的标注格式

        :param config: 相关配置，一般为 config['dataset']['train']['dataset] or config['dataset']['eval']['dataset]
                其主要应包含如下字段： file: 标注文件路径
                                    input_h: 图片的目标高
                                    mean: 归一化均值
                                    std: 归一化方差
                                    augmentation: 使用使用数据增强
        :return None
        """
        self.augmentation = config.augmentation
        self.process = RecDataProcess(config)
        self.str2idx = {c: i for i, c in enumerate(config.alphabet)}
        self.labels = []
        with open(config.file, 'r', encoding='utf-8') as f_reader:
            for m_line in f_reader.readlines():
                params = m_line.strip().split('\t')
                if len(params) == 2:
                    m_image_name, m_gt_text = params
                    print(m_gt_text)
                    if True in [c not in self.str2idx for c in m_gt_text]:
                        continue
                    self.labels.append((m_image_name, m_gt_text))

    def _find_max_length(self):
        return max({len(_[1]) for _ in self.labels})

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        # get img_path and trans
        img_path, trans = self.labels[index]
        # read img
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # do aug
        if self.augmentation:
            img = pil2cv(self.process.aug_img(cv2pil(img)))
        return {'img': img, 'label': trans}
    

class RecDataLoader:
    def __init__(self, dataset, batch_size, shuffle, num_workers, **kwargs):
        """
        自定义 DataLoader, 主要实现数据集的按长度划分，将长度相近的放在一个 batch

        :param dataset: 继承自 torch.utils.data.DataSet的类对象
        :param batch_size: 一个 batch 的图片数量
        :param shuffle: 是否打乱数据集
        :param num_workers: 后台进程数
        :param kwargs: **
        """
        self.dataset = dataset
        self.process = dataset.process
        self.len_thresh = self.dataset._find_max_length() // 2
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers
        self.iteration = 0
        self.dataiter = None
        self.queue_1 = list()
        self.queue_2 = list()

    def __len__(self):
        return len(self.dataset) // self.batch_size if len(self.dataset) % self.batch_size == 0 \
            else len(self.dataset) // self.batch_size + 1

    def __iter__(self):
        return self

    def pack(self, batch_data):
        batch = {'img': [], 'label': []}
        # img tensor current shape: B,H,W,C
        all_same_height_images = [self.process.resize_with_specific_height(_['img'][0].numpy()) for _ in batch_data]
        max_img_w = max({m_img.shape[1] for m_img in all_same_height_images})
        # make sure max_img_w is integral multiple of 8
        max_img_w = int(np.ceil(max_img_w / 8) * 8)
        for i in range(len(batch_data)):
            _label = batch_data[i]['label'][0]
            img = self.process.normalize_img(self.process.width_pad_img(all_same_height_images[i], max_img_w))
            img = img.transpose([2, 0, 1])
            batch['img'].append(torch.tensor(img, dtype=torch.float))
            batch['label'].append(_label)
        batch['img'] = torch.stack(batch['img'])
        return batch

    def build(self):
        self.dataiter = DataLoader(self.dataset, batch_size=1, shuffle=self.shuffle, num_workers=self.num_workers).__iter__()

    def __next__(self):
        if self.dataiter == None:
            self.build()
        if self.iteration == len(self.dataset) and len(self.queue_2):
            batch_data = self.queue_2
            self.queue_2 = list()
            return self.pack(batch_data)
        if not len(self.queue_2) and not len(self.queue_1) and self.iteration == len(self.dataset):
            self.iteration = 0
            self.dataiter = None
            raise StopIteration
        # start iteration
        try:
            while True:
                # get data from origin dataloader
                temp = self.dataiter.__next__()
                self.iteration += 1
                # to different queue
                if len(temp['label'][0]) <= self.len_thresh:
                    self.queue_1.append(temp)
                else:
                    self.queue_2.append(temp)

                # to store batch data
                batch_data = None
                # queue_1 full, push to batch_data
                if len(self.queue_1) == self.batch_size:
                    batch_data = self.queue_1
                    self.queue_1 = list()
                # or queue_2 full, push to batch_data
                elif len(self.queue_2) == self.batch_size:
                    batch_data = self.queue_2
                    self.queue_2 = list()

                # start to process batch
                if batch_data is not None:
                    return self.pack(batch_data)
        # deal with last batch
        except StopIteration:
            batch_data = self.queue_1
            self.queue_1 = list()
            return self.pack(batch_data)


class RecDataProcess:
    def __init__(self, config):
        """
        文本是被数据增广类

        :param config: 配置，主要用到的字段有 input_h, mean, std
        """
        self.config = config
        self.random_contrast = RandomContrast(probability=0.3)
        self.random_brightness = RandomBrightness(probability=0.3)
        self.random_sharpness = RandomSharpness(probability=0.3)
        self.compress = Compress(probability=0.3)
        self.rotate = Rotate(probability=0.5)
        self.blur = Blur(probability=0.3)
        self.motion_blur = MotionBlur(probability=0.3)
        self.salt = Salt(probability=0.3)
        self.adjust_resolution = AdjustResolution(probability=0.3)
        self.random_line = RandomLine(probability=0.3)
        self.random_contrast.setparam()
        self.random_brightness.setparam()
        self.random_sharpness.setparam()
        self.compress.setparam()
        self.rotate.setparam()
        self.blur.setparam()
        self.motion_blur.setparam()
        self.salt.setparam()
        self.adjust_resolution.setparam()

    def aug_img(self, img):
        img = self.random_contrast.process(img)
        img = self.random_brightness.process(img)
        img = self.random_sharpness.process(img)
        img = self.random_line.process(img)

        if img.size[1] >= 32:
            img = self.compress.process(img)
            img = self.adjust_resolution.process(img)
            img = self.motion_blur.process(img)
            img = self.blur.process(img)
        img = self.rotate.process(img)
        img = self.salt.process(img)
        return img

    def resize_with_specific_height(self, _img):
        """
        将图像resize到指定高度
        :param _img:    待resize的图像
        :return:    resize完成的图像
        """
        resize_ratio = self.config.input_h / _img.shape[0]
        return cv2.resize(_img, (0, 0), fx=resize_ratio, fy=resize_ratio, interpolation=cv2.INTER_LINEAR)

    def normalize_img(self, _img):
        """
        根据配置的均值和标准差进行归一化
        :param _img:    待归一化的图像
        :return:    归一化后的图像
        """
        return (_img.astype(np.float32) / 255 - self.config.mean) / self.config.std

    def width_pad_img(self, _img, _target_width, _pad_value=0):
        """
        将图像进行高度不变，宽度的调整的pad
        :param _img:    待pad的图像
        :param _target_width:   目标宽度
        :param _pad_value:  pad的值
        :return:    pad完成后的图像
        """
        _height, _width, _channels = _img.shape
        to_return_img = np.ones([_height, _target_width, _channels], dtype=_img.dtype) * _pad_value
        to_return_img[:_height, :_width, :] = _img
        return to_return_img


class RecCollateFn:
    def __init__(self, *args, **kwargs):
        self.process = kwargs['dataset'].process
        self.t = transforms.ToTensor()

    def __call__(self, batch):
        resize_images = []

        all_same_height_images = [self.process.resize_with_specific_height(_['img']) for _ in batch]
        max_img_w = max({m_img.shape[1] for m_img in all_same_height_images})
        # make sure max_img_w is integral multiple of 8
        max_img_w = int(np.ceil(max_img_w / 8) * 8)
        labels = []
        for i in range(len(batch)):
            _label = batch[i]['label']
            labels.append(_label)
            img = self.process.width_pad_img(all_same_height_images[i], max_img_w)
            img = self.process.normalize_img(img)
            img = img.transpose([2, 0, 1])
            resize_images.append(torch.tensor(img, dtype=torch.float))
        resize_images = torch.stack(resize_images)
        return {'img': resize_images, 'label': labels}

In [40]:
import os
import sys
import pathlib
import logging
logging.basicConfig(level = 20)

config_path = "/home/elimen/Data/dbnet_pytorch/config/rec_train_config.py"
module_name = os.path.basename(config_path)[:-3]
print(module_name)
config_dir = os.path.dirname(config_path)
print(config_dir)

rec_train_config
/home/elimen/Data/dbnet_pytorch/config


In [4]:
from importlib import import_module

sys.path.insert(0, config_dir)
mod = import_module(module_name)
sys.path.pop(0)

'/home/elimen/Data/dbnet_pytorch/config'

In [12]:
cfg = mod.config
cfg.dataset.alphabet = "/home/elimen/Data/dbnet_pytorch/torchocr/datasets/alphabets/ppocr_keys_v1.txt"

In [13]:
cfg

{'exp_name': 'CRNN',
 'train_options': {'resume_from': '',
  'third_party_name': '',
  'checkpoint_save_dir': './output/CRNN/checkpoint',
  'device': 'cuda:0',
  'epochs': 200,
  'fine_tune_stage': ['neck', 'head'],
  'print_interval': 10,
  'val_interval': 300,
  'ckpt_save_type': 'HighestAcc',
  'ckpt_save_epoch': 4},
 'SEED': 927,
 'optimizer': {'type': 'Adam', 'lr': 0.001, 'weight_decay': 0.0001},
 'lr_scheduler': {'type': 'StepLR', 'step_size': 60, 'gamma': 0.5},
 'model': {'type': 'RecModel',
  'backbone': {'type': 'ResNet', 'layers': 18},
  'neck': {'type': 'PPaddleRNN'},
  'head': {'type': 'CTC', 'n_class': 11},
  'in_channels': 3},
 'loss': {'type': 'CTCLoss', 'blank_idx': 0},
 'dataset': {'alphabet': '/home/elimen/Data/dbnet_pytorch/torchocr/datasets/alphabets/ppocr_keys_v1.txt',
  'train': {'dataset': {'type': 'RecTextLineDataset',
    'file': '/home/elimen/Data/OCR_dataset/icdar2015/recognition/train.txt',
    'input_h': 32,
    'mean': 0.5,
    'std': 0.5,
    'augmentatio

In [14]:
with open(cfg.dataset.alphabet, 'r', encoding='utf-8') as file:
    cfg.dataset.alphabet = ''.join([s.strip('\n') for s in file.readlines()])
cfg.dataset.alphabet

'\'疗绚诚娇溜题贿者廖更纳加奉公一就汴计与路房原妇208-7其>:],，骑刈全消昏傈安久钟嗅不影处驽蜿资关椤地瘸专问忖票嫉炎韵要月田节陂鄙捌备拳伺眼网盎大傍心东愉汇蹿科每业里航晏字平录先13彤鲶产稍督腴有象岳注绍在泺文定核名水过理让偷率等这发”为含肥酉相鄱七编猥锛日镀蒂掰倒辆栾栗综涩州雌滑馀了机块司宰甙兴矽抚保用沧秩如收息滥页疑埠!！姥异橹钇向下跄的椴沫国绥獠报开民蜇何分凇长讥藏掏施羽中讲派嘟人提浼间世而古多倪唇饯控庚首赛蜓味断制觉技替艰溢潮夕钺外摘枋动双单啮户枇确锦曜杜或能效霜盒然侗电晁放步鹃新杖蜂吒濂瞬评总隍对独合也是府青天诲墙组滴级邀帘示已时骸仄泅和遨店雇疫持巍踮境只亨目鉴崤闲体泄杂作般轰化解迂诿蛭璀腾告版服省师小规程线海办引二桧牌砺洄裴修图痫胡许犊事郛基柴呼食研奶律蛋因葆察戏褒戒再李骁工貂油鹅章啄休场给睡纷豆器捎说敏学会浒设诊格廓查来霓室溆￠诡寥焕舜柒狐回戟砾厄实翩尿五入径惭喹股宇篝|;美期云九祺扮靠锝槌系企酰阊暂蚕忻豁本羹执条钦H獒限进季楦于芘玖铋茯未答粘括样精欠矢甥帷嵩扣令仔风皈行支部蓉刮站蜡救钊汗松嫌成可.鹤院从交政怕活调球局验髌第韫谗串到圆年米/*友忿检区看自敢刃个兹弄流留同没齿星聆轼湖什三建蛔儿椋汕震颧鲤跟力情璺铨陪务指族训滦鄣濮扒商箱十召慷辗所莞管护臭横硒嗓接侦六露党馋驾剖高侬妪幂猗绺骐央酐孝筝课徇缰门男西项句谙瞒秃篇教碲罚声呐景前富嘴鳌稀免朋啬睐去赈鱼住肩愕速旁波厅健茼厥鲟谅投攸炔数方击呋谈绩别愫僚躬鹧胪炳招喇膨泵蹦毛结54谱识陕粽婚拟构且搜任潘比郢妨醪陀桔碘扎选哈骷楷亿明缆脯监睫逻婵共赴淝凡惦及达揖谩澹减焰蛹番祁柏员禄怡峤龙白叽生闯起细装谕竟聚钙上导渊按艾辘挡耒盹饪臀记邮蕙受各医搂普滇朗茸带翻酚(光堤墟蔷万幻〓瑙辈昧盏亘蛀吉铰请子假闻税井诩哨嫂好面琐校馊鬣缂营访炖占农缀否经钚棵趟张亟吏茶谨捻论迸堂玉信吧瞠乡姬寺咬溏苄皿意赉宝尔钰艺特唳踉都荣倚登荐丧奇涵批炭近符傩感道着菊虹仲众懈濯颞眺南释北缝标既茗整撼迤贲挎耱拒某妍卫哇英矶藩治他元领膜遮穗蛾飞荒棺劫么市火温拈棚洼转果奕卸迪伸泳斗邡侄涨屯萋胭氡崮枞惧冒彩斜手豚随旭淑妞形菌吲沱争驯歹挟兆柱传至包内响临红功弩衡寂禁老棍耆渍织害氵渑布载靥嗬虽苹咨娄库雉榜帜嘲套瑚亲簸欧边6腿旮抛吹瞳得镓梗厨继漾愣憨士策窑抑躯襟脏参贸言干绸鳄穷藜音折详)举悍甸癌黎谴死罩迁寒驷袖媒蒋掘模纠恣观祖蛆碍位稿主澧跌筏京锏帝贴证

In [15]:
cfg.dataset.train.dataset.alphabet = cfg.dataset.alphabet

In [16]:
cfg

{'exp_name': 'CRNN',
 'train_options': {'resume_from': '',
  'third_party_name': '',
  'checkpoint_save_dir': './output/CRNN/checkpoint',
  'device': 'cuda:0',
  'epochs': 200,
  'fine_tune_stage': ['neck', 'head'],
  'print_interval': 10,
  'val_interval': 300,
  'ckpt_save_type': 'HighestAcc',
  'ckpt_save_epoch': 4},
 'SEED': 927,
 'optimizer': {'type': 'Adam', 'lr': 0.001, 'weight_decay': 0.0001},
 'lr_scheduler': {'type': 'StepLR', 'step_size': 60, 'gamma': 0.5},
 'model': {'type': 'RecModel',
  'backbone': {'type': 'ResNet', 'layers': 18},
  'neck': {'type': 'PPaddleRNN'},
  'head': {'type': 'CTC', 'n_class': 11},
  'in_channels': 3},
 'loss': {'type': 'CTCLoss', 'blank_idx': 0},
 'dataset': {'alphabet': '\'疗绚诚娇溜题贿者廖更纳加奉公一就汴计与路房原妇208-7其>:],，骑刈全消昏傈安久钟嗅不影处驽蜿资关椤地瘸专问忖票嫉炎韵要月田节陂鄙捌备拳伺眼网盎大傍心东愉汇蹿科每业里航晏字平录先13彤鲶产稍督腴有象岳注绍在泺文定核名水过理让偷率等这发”为含肥酉相鄱七编猥锛日镀蒂掰倒辆栾栗综涩州雌滑馀了机块司宰甙兴矽抚保用沧秩如收息滥页疑埠!！姥异橹钇向下跄的椴沫国绥獠报开民蜇何分凇长讥藏掏施羽中讲派嘟人提浼间世而古多倪唇饯控庚首赛蜓味断制觉技替艰溢潮夕钺外摘枋动双单啮户枇确锦曜杜或能效霜盒然侗电晁放步鹃新杖蜂吒濂瞬评总隍对独合也是府青天诲墙组滴级邀帘示已时骸仄

In [20]:
import copy
from addict import Dict
train_config = cfg.dataset.train
train_config_copy = copy.deepcopy(train_config)
print(train_config_copy)

{'dataset': {'type': 'RecTextLineDataset', 'file': '/home/elimen/Data/OCR_dataset/icdar2015/recognition/train.txt', 'input_h': 32, 'mean': 0.5, 'std': 0.5, 'augmentation': False, 'alphabet': '\'疗绚诚娇溜题贿者廖更纳加奉公一就汴计与路房原妇208-7其>:],，骑刈全消昏傈安久钟嗅不影处驽蜿资关椤地瘸专问忖票嫉炎韵要月田节陂鄙捌备拳伺眼网盎大傍心东愉汇蹿科每业里航晏字平录先13彤鲶产稍督腴有象岳注绍在泺文定核名水过理让偷率等这发”为含肥酉相鄱七编猥锛日镀蒂掰倒辆栾栗综涩州雌滑馀了机块司宰甙兴矽抚保用沧秩如收息滥页疑埠!！姥异橹钇向下跄的椴沫国绥獠报开民蜇何分凇长讥藏掏施羽中讲派嘟人提浼间世而古多倪唇饯控庚首赛蜓味断制觉技替艰溢潮夕钺外摘枋动双单啮户枇确锦曜杜或能效霜盒然侗电晁放步鹃新杖蜂吒濂瞬评总隍对独合也是府青天诲墙组滴级邀帘示已时骸仄泅和遨店雇疫持巍踮境只亨目鉴崤闲体泄杂作般轰化解迂诿蛭璀腾告版服省师小规程线海办引二桧牌砺洄裴修图痫胡许犊事郛基柴呼食研奶律蛋因葆察戏褒戒再李骁工貂油鹅章啄休场给睡纷豆器捎说敏学会浒设诊格廓查来霓室溆￠诡寥焕舜柒狐回戟砾厄实翩尿五入径惭喹股宇篝|;美期云九祺扮靠锝槌系企酰阊暂蚕忻豁本羹执条钦H獒限进季楦于芘玖铋茯未答粘括样精欠矢甥帷嵩扣令仔风皈行支部蓉刮站蜡救钊汗松嫌成可.鹤院从交政怕活调球局验髌第韫谗串到圆年米/*友忿检区看自敢刃个兹弄流留同没齿星聆轼湖什三建蛔儿椋汕震颧鲤跟力情璺铨陪务指族训滦鄣濮扒商箱十召慷辗所莞管护臭横硒嗓接侦六露党馋驾剖高侬妪幂猗绺骐央酐孝筝课徇缰门男西项句谙瞒秃篇教碲罚声呐景前富嘴鳌稀免朋啬睐去赈鱼住肩愕速旁波厅健茼厥鲟谅投攸炔数方击呋谈绩别愫僚躬鹧胪炳招喇膨泵蹦毛结54谱识陕粽婚拟构且搜任潘比郢妨醪陀桔碘扎选哈骷楷亿明缆脯监睫逻婵共赴淝凡惦及达揖谩澹减焰蛹番祁柏员禄怡峤龙白叽生闯起细装谕竟聚钙上导渊按艾辘挡耒盹饪臀记邮蕙受各医搂普滇朗茸带翻酚(光堤墟蔷万幻〓瑙辈昧盏亘蛀吉铰请子假闻税井诩哨嫂好面琐校馊鬣缂营访炖占农缀否经钚棵趟张亟吏茶谨捻论迸堂玉信吧瞠乡姬寺咬溏苄皿意赉宝尔钰艺特唳踉都荣倚登荐丧奇涵批炭近符傩感道

In [21]:
train_config_copy.dataset

{'type': 'RecTextLineDataset',
 'file': '/home/elimen/Data/OCR_dataset/icdar2015/recognition/train.txt',
 'input_h': 32,
 'mean': 0.5,
 'std': 0.5,
 'augmentation': False,
 'alphabet': '\'疗绚诚娇溜题贿者廖更纳加奉公一就汴计与路房原妇208-7其>:],，骑刈全消昏傈安久钟嗅不影处驽蜿资关椤地瘸专问忖票嫉炎韵要月田节陂鄙捌备拳伺眼网盎大傍心东愉汇蹿科每业里航晏字平录先13彤鲶产稍督腴有象岳注绍在泺文定核名水过理让偷率等这发”为含肥酉相鄱七编猥锛日镀蒂掰倒辆栾栗综涩州雌滑馀了机块司宰甙兴矽抚保用沧秩如收息滥页疑埠!！姥异橹钇向下跄的椴沫国绥獠报开民蜇何分凇长讥藏掏施羽中讲派嘟人提浼间世而古多倪唇饯控庚首赛蜓味断制觉技替艰溢潮夕钺外摘枋动双单啮户枇确锦曜杜或能效霜盒然侗电晁放步鹃新杖蜂吒濂瞬评总隍对独合也是府青天诲墙组滴级邀帘示已时骸仄泅和遨店雇疫持巍踮境只亨目鉴崤闲体泄杂作般轰化解迂诿蛭璀腾告版服省师小规程线海办引二桧牌砺洄裴修图痫胡许犊事郛基柴呼食研奶律蛋因葆察戏褒戒再李骁工貂油鹅章啄休场给睡纷豆器捎说敏学会浒设诊格廓查来霓室溆￠诡寥焕舜柒狐回戟砾厄实翩尿五入径惭喹股宇篝|;美期云九祺扮靠锝槌系企酰阊暂蚕忻豁本羹执条钦H獒限进季楦于芘玖铋茯未答粘括样精欠矢甥帷嵩扣令仔风皈行支部蓉刮站蜡救钊汗松嫌成可.鹤院从交政怕活调球局验髌第韫谗串到圆年米/*友忿检区看自敢刃个兹弄流留同没齿星聆轼湖什三建蛔儿椋汕震颧鲤跟力情璺铨陪务指族训滦鄣濮扒商箱十召慷辗所莞管护臭横硒嗓接侦六露党馋驾剖高侬妪幂猗绺骐央酐孝筝课徇缰门男西项句谙瞒秃篇教碲罚声呐景前富嘴鳌稀免朋啬睐去赈鱼住肩愕速旁波厅健茼厥鲟谅投攸炔数方击呋谈绩别愫僚躬鹧胪炳招喇膨泵蹦毛结54谱识陕粽婚拟构且搜任潘比郢妨醪陀桔碘扎选哈骷楷亿明缆脯监睫逻婵共赴淝凡惦及达揖谩澹减焰蛹番祁柏员禄怡峤龙白叽生闯起细装谕竟聚钙上导渊按艾辘挡耒盹饪臀记邮蕙受各医搂普滇朗茸带翻酚(光堤墟蔷万幻〓瑙辈昧盏亘蛀吉铰请子假闻税井诩哨嫂好面琐校馊鬣缂营访炖占农缀否经钚棵趟张亟吏茶谨捻论迸堂玉信吧瞠乡姬寺咬溏苄皿意赉宝尔钰艺特唳踉都荣倚登荐丧奇涵批炭近符傩感道着菊虹仲众懈

In [24]:
train_dataset_type = train_config_copy.dataset['type']
train_dataset_type

'RecTextLineDataset'

In [137]:
train_dataset = RecTextLineDataset(train_config_copy.dataset)
train_dataset

1
Genaxis Theatre
[06]
62-03
Carpark
EXIT
I2R
fusionopolis
fusionopolis
Reserve
CAUTION
citi
smrt
WHY
PAY
FOR
NOTHING?
EXIT
STAGE
HarbourFront
CC22
bua
Shops
Gen is
Theatre
Place
Nursing
Room
Fraser
LIFESTYLE
CHOICES
THAT
FOR
COME
SHOP
DINE
IPAL
IPAL
Stude
$500
$4.00
$4.00
$4.00
Lower
colorie
DUMPING
STRICTLY
PROHIBITED
STOREY
LOBBY
LIFT
Clementi
Service
Mall
173
166
175
285
285
Add
Value
Machine
Clementi
The
Pasenger
Toilet
BERTH
0000
who
needs
nela
PULL
PULL
THE
WORLD
YOU
BRING
Have
NUS
Share
wiht
AGSC2015
YOUR
NOTES
TRIALS
BUS
rale
SELL
NUS
NUS
Faculty
Engine
Internal
Shuttle
Bus
MRT
Kent
Ridge
ONE
LEVEL
DOWN
LOVE
SMRT
Straits
Optical
Phoon
Huat
Since
!947
Baking
Made
Easy
BAKING
MADE
EASY
MARIGOLD
City
Singapura
Ngee
Ann
City
ION
Orchard
Paragon
Plaze
Raffles
Suntec
City
TANGS
197
Occitane
PROVENCE
VICTORIA
SMOKING
EARTH
TRAIL
HENDERSON
WAVES
Bus
Interchange
Shop
Dine
SMRT
ice
HAS
YOUR
LOVED
BEEN
ONE
times
minute;
know
that
the
you
average
heart
adult
beats
000
times
day;
000
times

ESPRIT
DBS
REHOUSE
Calvin
Calvin
CITIGEMS
EMPORIO
TAG
ARMANI
MAKE
SEIKO
MAE
MAK
L'OCCITANE
MAKE
ORE
ASTRON
Lou
Yau
=B2-25
Superdry.
NovOpe
Level1
- #01
asian
ZTP
KOI
SNAFFLES
SNAFFLES
B2-K15
week
1KG
my4
LOSE
simple
steps
the
delight
here.
000
UOB
Joy
Dare
Comparel
Compare!
Flowers
Cut
VALUE
BUY
4.20
3.15
3.15
RAND
LOW
PRICES
DAILY
CUBES
VALUE
ICE
CUBES
CUBES
FOR
WALL'S
WALL
DRINKS
ICE
CUBES
DHA
Nature
CRIME
and
Long-lasting
Color
improved
creamy
foam
with
AHITARGYM
SHOP
where
people
meet
Massage
Acupuncture
Massotherapy
Body
Massage
CHANGER
MONEY
CHAN
OTO
OTO
Life
ichiban
boshi
lehiban
twelve
cup
handmade
aucre
BROWNIES
LOAVES
TARTS
PARFAITS
PUDDINGS
PUSH
WHOOPIES
CITIGEMS
CANDY
EMPIRE
TOASTBOX
Standard
ANK
UOB
DON
DON
prettyFIT
BreadTalk
VEIL
mds
VRG
FRIENDS
MAKE
re           lles
ETT
CiTYLiNK
AEROPOSTALE
LAST
DAY
JEANS
B  ATA
CONNECTION
EXIT
BODY
SHOP
IT'S
BIG
THE
SALE
KISSJane
fipper
Feel
the
Rubber!
outhaven
ThaiExpress
SALADSTOP!
Square
CiTYLiNK
LOOK
OUT
FOR
BRAND
NEW
RETAIL
WING

In [138]:
train_config_copy.loader

{'type': 'DataLoader',
 'batch_size': 16,
 'shuffle': True,
 'num_workers': 1,
 'collate_fn': {'type': 'RecCollateFn',
  'img_w': 120,
  'dataset': <__main__.RecTextLineDataset at 0x7f1060adac90>}}

In [35]:
train_dataloader_type = 'DataLoader'
train_collate_fn_dict = {'type': 'RecCollateFn', 'img_w': 120}
train_collate_fn_type = 'RecCollateFn'

In [139]:
train_config_copy.loader.collate_fn['dataset'] = train_dataset

In [140]:
train_config_copy.loader

{'type': 'DataLoader',
 'batch_size': 16,
 'shuffle': True,
 'num_workers': 1,
 'collate_fn': {'type': 'RecCollateFn',
  'img_w': 120,
  'dataset': <__main__.RecTextLineDataset at 0x7f10462f2c10>}}

In [141]:
train_collate_fn = RecCollateFn(**(train_config_copy.loader.collate_fn))

In [142]:
type(train_collate_fn)
train_config_copy.loader

{'type': 'DataLoader',
 'batch_size': 16,
 'shuffle': True,
 'num_workers': 1,
 'collate_fn': {'type': 'RecCollateFn',
  'img_w': 120,
  'dataset': <__main__.RecTextLineDataset at 0x7f10462f2c10>}}

In [176]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size = 2, shuffle = True, num_workers = 2)

In [178]:
dataiter = iter(train_dataloader)

In [179]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [180]:
next(dataiter)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/elimen/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/elimen/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/elimen/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 73, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in elem}
  File "/home/elimen/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 73, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in elem}
  File "/home/elimen/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 63, in default_collate
    return

In [171]:
label_list=[]
dirpath = "/home/elimen/Data/OCR_dataset"
with open("/home/elimen/Data/OCR_dataset/icdar2015/recognition/train_bak.txt", 'r', encoding='utf-8') as f_reader:
    for m_line in f_reader.readlines():
        params = m_line.strip().split('\t')
        if len(params) == 3:
            m_image_name, m_gt_text, _ = params
            path_split = m_image_name.split('\\')[2:]
            path = ''
            for s in path_split:
                path = os.path.join(path,s)
            img_path = os.path.join(dirpath,path)
            label = img_path + "\t" + m_gt_text
            label_list.append(label)
        else:
            print("Wrong content:{}".format(params))

In [175]:
with open("train.txt","w") as f_writer:
    for i in range(len(label_list)):
        f_writer.writelines(label_list[i]+"\n")

In [183]:
label_list=[]
dirpath = "/home/elimen/Data/OCR_dataset"
with open("/home/elimen/Data/OCR_dataset/icdar2015/recognition/test_bak.txt", 'r', encoding='utf-8') as f_reader:
    for m_line in f_reader.readlines():
        params = m_line.strip().split('\t')
        if len(params) == 3:
            m_image_name, m_gt_text, _ = params
            path_split = m_image_name.split('\\')[2:]
            path = ''
            for s in path_split:
                path = os.path.join(path,s)
            img_path = os.path.join(dirpath,path)
            label = img_path + "\t" + m_gt_text
            label_list.append(label)
        else:
            print("Wrong content:{}".format(params))

In [184]:
with open("eval.txt","w") as f_writer:
    for i in range(len(label_list)):
        f_writer.writelines(label_list[i]+"\n")

In [194]:
import json
with open('/home/elimen/Data/OCR_dataset/synthetic_digit/labels.json', 'r') as f_json:
    content = f_json.read()
digitimg_dict = json.loads(content)
digitimg_dict['labels']

{'000000000': ',580,368.722,',
 '000000001': ',724,282.336,',
 '000000002': ',945.608,012,',
 '000000003': '3,856,048.814',
 '000000004': ',232,981,268.',
 '000000005': '42,346.461,11',
 '000000006': '9.646,629,956',
 '000000007': '75,240,443,83',
 '000000008': '9,375,392.397',
 '000000009': '05,112,130,96',
 '000000010': '61,653,981.73',
 '000000011': '26.428,765,58',
 '000000012': '717,692,038,1',
 '000000013': '51.419,846,58',
 '000000014': '.607,746,369,',
 '000000015': ',572.948,725,',
 '000000016': '.508,037,705,',
 '000000017': '3306,790,159.',
 '000000018': '2,377.691,569',
 '000000019': '9418,899,237.',
 '000000020': '508,037,705,4',
 '000000021': ',203.331,573,',
 '000000022': ',895,980.357,',
 '000000023': '0,117,306.657',
 '000000024': '6,465,362,910',
 '000000025': '2,312.846,929',
 '000000026': '409,973.619,0',
 '000000027': '978,115,340,7',
 '000000028': '98,779,464.67',
 '000000029': '78,504,087.65',
 '000000030': ',314,224.8480',
 '000000031': '214,979,835.3',
 '000000

In [203]:
label_list=[]
dirpath = "/home/elimen/Data/OCR_dataset/synthetic_digit"

for i,k in enumerate(digitimg_dict['labels']):
    v = digitimg_dict['labels'][k]
    path = ''
    img_path = os.path.join(dirpath,d)
    label = img_path + ".jpg" + "\t" + v
    label_list.append(label)

label_list

['/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t,580,368.722,',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t,724,282.336,',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t,945.608,012,',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t3,856,048.814',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t,232,981,268.',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t42,346.461,11',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t9.646,629,956',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t75,240,443,83',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t9,375,392.397',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t05,112,130,96',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t61,653,981.73',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/000000000.jpg\t26.428,765,58',
 '/home/elimen/Data/OCR_dataset/synthetic_digit/0000

In [204]:
with open("/home/elimen/Data/OCR_dataset/synthetic_digit/train.txt","w") as fjson_writer:
    for i in range(len(label_list)):
        fjson_writer.writelines(label_list[i]+"\n")